In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
np.random.seed(2018)

In [2]:
#データの呼び出し
trn = pd.read_csv("santander-product-recommendation/train_ver2.csv")
tst = pd.read_csv("santander-product-recommendation/test_ver2.csv")

c:\Users\astro\anaconda3\envs\kaggle_conda\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\Users\astro\anaconda3\envs\kaggle_conda\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
prods = trn.columns[24:].tolist()

#商品変数のnanを0に代替(fillna)
trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

In [4]:
#24個の商品を一個も持っていない顧客のデータを除去
no_product = trn[prods].sum(axis=1) == 0
"""
== 0 でsumの結果が0である行(顧客)はTrueとなるSeriesになる．
df[~bool]でFalseのみ残すことができる(df[bool]の逆)
"""

'\n== 0 でsumの結果が0である行(顧客)はTrueとなるSeriesになる．\ndf[~bool]でFalseのみ残すことができる(df[bool]の逆)\n'

In [5]:
trn = trn[~no_product]

In [6]:
trn.shape

(11091070, 48)

In [7]:
#訓練データとテストデータを統合．テストデータにない商品変数は0
for col in trn.columns[24:]:
    tst[col] = 0
df = pd.concat([trn,tst],axis=0)


In [8]:
feautures = []

categorical_cols = ["ind_empleado","pais_residencia","sexo","tiprel_1mes","indresi","indext","conyuemp","canal_entrada","indfall","tipodom","nomprov","segmento"]

In [9]:
for col in categorical_cols:
    df[col],uni = df[col].factorize(na_sentinel=-99)

In [10]:
feautures += categorical_cols

In [11]:
df["age"].replace(" NA",-99,inplace=True)
df["age"] = df["age"].astype(np.int8)

In [12]:
df["antiguedad"].replace("     NA",-99,inplace=True)
df["antiguedad"] = df["antiguedad"].astype(np.int8)

In [13]:
df["renta"].replace("         NA",-99,inplace=True)
df["renta"].fillna(-99,inplace=True)
df["renta"] = df["renta"].astype(float).astype(np.int8)

In [14]:
df["indrel_1mes"].replace("P",-99,inplace=True)
df["indrel_1mes"].fillna(-99,inplace=True)
df["indrel_1mes"] = df["indrel_1mes"].astype(float).astype(np.int8)

In [15]:
#学習に使用する数値型変数をfeaturesに追加
feautures += ["age","antiguedad","renta","ind_nuevo","indrel","indrel_1mes","ind_actividad_cliente"]

In [16]:
#2つの日付変数から年度と月の情報を抽出
#(2022-01-01 ->split("-")-> [2020,01,01])
df["fecha_alta_month"] = df["fecha_alta"].map(lambda x: 0.0 if x.__class__ is float else float(x.split("-")[1])).astype(np.int8)
df["fecha_alta_year"]  = df["fecha_alta"].map(lambda x: 0.0 if x.__class__ is float else float(x.split("-")[0])).astype(np.int8)
feautures += ["fecha_alta_month","fecha_alta_year"]

In [17]:
df["ult_fec_cli_1t_month"] = df["ult_fec_cli_1t"].map(lambda x: 0.0 if x.__class__ is float else float(x.split("-")[1])).astype(np.int8)
df["ult_fec_cli_1t_year"] = df["ult_fec_cli_1t"].map(lambda x: 0.0 if x.__class__ is float else float(x.split("-")[0])).astype(np.int8)
feautures += ["ult_fec_cli_1t_month","ult_fec_cli_1t_year"]

In [18]:
df.fillna(-99,inplace=True)

In [19]:
#日付を数字に変換する関数
def date_to_int(str_date:str)->int:
    Y,M,D = map(int,str_date.strip().split("-"))
    int_date = (Y - 2015) * 12 + M
    return int_date

In [20]:
#日付を数字に変換し，"int_date"として保存
df["int_date"] = df["fecha_dato"].map(date_to_int).astype(np.int8)

In [21]:
df_lag = df.copy()
df_lag.columns = [col + "_prev" if col not in ["ncodpers","int_date"] else col for col in df.columns]
df_lag["int_date"] += 1

In [22]:
# df_lag.to_csv("df_lag.csv")

In [ ]:
df_trn = df.merge(df_lag,on=["ncodpers","int_date"],how="left")
del df,df_lag

In [ ]:
for prod in prods:
    prev = prod + "_prev"
    df_trn[prev].fillna(0,inplace=True)
df_trn.fillna(-99,inplace=True)

In [ ]:
feautures += [feature + "_prev" for feature in feautures]
feautures += [prod + "prev" for prod in prods]